# Driven Data Competition # 
## Flu Shot Learning: Predict H1N1 and Seasonal Flu Vaccines ##
https://www.drivendata.org/competitions/66/flu-shot-learning/page/210/

### Problem Description ###
---
Your goal is to predict how likely individuals are to receive their H1N1 and seasonal flu vaccines. Specifically, you'll be predicting two probabilities: one for `h1n1_vaccine` and one for `seasonal_vaccine`.

Each row in the dataset represents one person who responded to the National 2009 H1N1 Flu Survey.

### Labels ###
---
For this competition, there are two target variables:

- `h1n1_vaccine` - Whether respondent received H1N1 flu vaccine.
- `seasonal_vaccine` - Whether respondent received seasonal flu vaccine.

Both are binary variables: `0` = No; `1` = Yes. Some respondents didn't get either vaccine, others got only one, and some got both. This is formulated as a multilabel (and not multiclass) problem.

### Features in this Dataset ###
---
You are provided a dataset with 36 columns. The first column `respondent_id` is a unique and random identifier. The remaining 35 features are described below.

For all binary variables: `0` = No; `1` = Yes.

- `h1n1_concern` - Level of concern about the H1N1 flu.
    - `0` = Not at all concerned; `1` = Not very concerned; `2` = Somewhat concerned; `3` = Very concerned.
- `h1n1_knowledge` - Level of knowledge about H1N1 flu.
    - `0` = No knowledge; `1` = A little knowledge; `2` = A lot of knowledge.
- `behavioral_antiviral_meds` - Has taken antiviral medications. (binary)
- `behavioral_avoidance` - Has avoided close contact with others with flu-like symptoms. (binary)
- `behavioral_face_mask` - Has bought a face mask. (binary)
- `behavioral_wash_hands` - Has frequently washed hands or used hand sanitizer. (binary)
- `behavioral_large_gatherings` - Has reduced time at large gatherings. (binary)
- `behavioral_outside_home` - Has reduced contact with people outside of own household. (binary)
- `behavioral_touch_face` - Has avoided touching eyes, nose, or mouth. (binary)
- `doctor_recc_h1n1` - H1N1 flu vaccine was recommended by doctor. (binary)
- `doctor_recc_seasonal` - Seasonal flu vaccine was recommended by doctor. (binary)
- `chronic_med_condition` - Has any of the following chronic medical conditions: asthma or an other lung condition, diabetes, a heart condition, a kidney condition, sickle cell anemia or other anemia, a neurological or neuromuscular condition, a liver condition, or a weakened immune system caused by a chronic illness or by medicines taken for a chronic illness. (binary)
- `child_under_6_months` - Has regular close contact with a child under the age of six months. (binary)
- `health_worker` - Is a healthcare worker. (binary)
- `health_insurance` - Has health insurance. (binary)
- `opinion_h1n1_vacc_effective` - Respondent's opinion about H1N1 vaccine effectiveness.
    - `1` = Not at all effective; `2` = Not very effective; `3` = Don't know; `4` = Somewhat effective; `5` = Very effective.
- `opinion_h1n1_risk` - Respondent's opinion about risk of getting sick with H1N1 flu without vaccine.
    - `1` = Very Low; `2` = Somewhat low; `3` = Don't know; `4` = Somewhat high; `5` = Very high.
- `opinion_h1n1_sick_from_vacc` - Respondent's worry of getting sick from taking H1N1 vaccine.
    - `1` = Not at all worried; `2` = Not very worried; `3` = Don't know; `4` = Somewhat worried; `5` = Very worried.
- `opinion_seas_vacc_effective` - Respondent's opinion about seasonal flu vaccine effectiveness.
    - `1` = Not at all effective; `2` = Not very effective; `3` = Don't know; `4` = Somewhat effective; `5` = Very effective.
- `opinion_seas_risk` - Respondent's opinion about risk of getting sick with seasonal flu without vaccine.
    - `1` = Very Low; `2` = Somewhat low; `3` = Don't know; `4` = Somewhat high; `5` = Very high.
- `opinion_seas_sick_from_vacc` - Respondent's worry of getting sick from taking seasonal flu vaccine.
    - `1` = Not at all worried; `2` = Not very worried; `3` = Don't know; `4` = Somewhat worried; `5` = Very worried.
- `age_group` - Age group of respondent.
- `education` - Self-reported education level.
- `race` - Race of respondent.
- `sex` - Sex of respondent.
- `income_poverty` - Household annual income of respondent with respect to 2008 Census poverty thresholds.
- `marital_status` - Marital status of respondent.
- `rent_or_own` - Housing situation of respondent.
- `employment_status` - Employment status of respondent.
- `hhs_geo_region` - Respondent's residence using a 10-region geographic classification defined by the U.S. Dept. of Health and Human Services. Values are represented as short random character strings.
- `census_msa` - Respondent's residence within metropolitan statistical areas (MSA) as defined by the U.S. Census.
- `household_adults` - Number of other adults in household, top-coded to 3.
- `household_children` - Number of children in household, top-coded to 3.
- `employment_industry` - Type of industry respondent is employed in. Values are represented as short random character strings.
- `employment_occupation` - Type of occupation of respondent. Values are represented as short random character strings.

### Performance Metric ###
---
Performance will be evaluated according to the area under the receiver operating characteristic curve (ROC AUC) for each of the two target variables. The mean of these two scores will be the overall score. A higher value indicates stronger performance.

In Python, you can calculate this using `sklearn.metrics.roc_auc_score` for this multilabel setup with the default `average="macro"` parameter.

### Submission Format ###
---
The format for the submission file is three columns: `respondent_id`, `h1n1_vaccine`, and `seasonal_vaccine`. The predictions for the two target variables should be <b>float probabilities</b> that range between `0.0` and `1.0`. Because the competition uses ROC AUC as its evaluation metric, the values you submit must be the probabilities that a person received each vaccine, not binary labels.

As this is a multilabel problem, the probabilities for each row do not need to sum to one.

# Let's Begin! #
## Start with loading in and exploring data ##

In [1]:
# from __future__ import division
# from __future__ import print_function

# ignore deprecation warnings in sklearn
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import os
import sys

# add the 'src' directory as one where we can import modules
src_dir = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)

# Similar to hard coding this but dynamic (so you can copy and paste, or clone this project repository)
# path = '/Users/bpolzin/Documents/driven_data/predicting_vaccines/src'
# sys.path.append(path)

from data.multilabel import multilabel_sample_dataframe, multilabel_train_test_split
from features.SparseInteractions import SparseInteractions
# from models.metrics import multi_multi_log_loss

# Add the simple imputer 
from sklearn.impute import SimpleImputer

In [2]:
# Load in our data
df_train_features = pd.read_csv('../vaccine_data/training_set_features.csv')
df_train_labels = pd.read_csv('../vaccine_data/training_set_labels.csv')
df_test_features = pd.read_csv('../vaccine_data/test_set_features.csv')

In [3]:
df_train_features.head(5)

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb


In [4]:
df_train_labels.head(5)

,respondent_id,h1n1_vaccine,seasonal_vaccine
0,0,0,0
1,1,0,1
2,2,0,0
3,3,0,1
4,4,0,0


In [5]:
df_train_labels.groupby(['h1n1_vaccine','seasonal_vaccine']).agg('count')

respondent_id
h1n1_vaccine seasonal_vaccine               
0            0                         13295
             1                          7738
1            0                           977
             1                          4697

In [6]:
df_train_labels['h1n1_vaccine'].value_counts()

0    21033
1     5674
Name: h1n1_vaccine, dtype: int64

In [7]:
df_train_labels['seasonal_vaccine'].value_counts()

0    14272
1    12435
Name: seasonal_vaccine, dtype: int64

In [8]:
df_train_labels.describe()

,respondent_id,h1n1_vaccine,seasonal_vaccine
count,26707.000000,26707.000000,26707.000000
mean,13353.000000,0.212454,0.465608
std,7709.791156,0.409052,0.498825
min,0.000000,0.000000,0.000000
25%,6676.500000,0.000000,0.000000
50%,13353.000000,0.000000,0.000000
75%,20029.500000,0.000000,1.000000
max,26706.000000,1.000000,1.000000


In [9]:
df_test_features.head(5)

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,26707,2.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,"> $75,000",Not Married,Rent,Employed,mlyzmhmf,"MSA, Not Principle City",1.0,0.0,atmlpfrs,hfxkjkmi
1,26708,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,Non-MSA,3.0,0.0,atmlpfrs,xqwwgdyp
2,26709,2.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,"> $75,000",Married,Own,Employed,lrircsnp,Non-MSA,1.0,0.0,nduyfdeo,pvmttkik
3,26710,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Married,Own,Not in Labor Force,lrircsnp,"MSA, Not Principle City",1.0,0.0,NaN,NaN
4,26711,3.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,lzgpxyit,Non-MSA,0.0,1.0,fcxhlnwr,mxkfnird


### Going to save the index and respondent_id as a df to id them later ###

In [10]:
df_test_ids = pd.DataFrame(df_test_features['respondent_id'])

In [11]:
df_train_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26707 entries, 0 to 26706
Data columns (total 36 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   respondent_id                26707 non-null  int64  
 1   h1n1_concern                 26615 non-null  float64
 2   h1n1_knowledge               26591 non-null  float64
 3   behavioral_antiviral_meds    26636 non-null  float64
 4   behavioral_avoidance         26499 non-null  float64
 5   behavioral_face_mask         26688 non-null  float64
 6   behavioral_wash_hands        26665 non-null  float64
 7   behavioral_large_gatherings  26620 non-null  float64
 8   behavioral_outside_home      26625 non-null  float64
 9   behavioral_touch_face        26579 non-null  float64
 10  doctor_recc_h1n1             24547 non-null  float64
 11  doctor_recc_seasonal         24547 non-null  float64
 12  chronic_med_condition        25736 non-null  float64
 13  child_under_6_mo

In [12]:
# Explore df_train_features for duplicated columns
df_train_features.duplicated().sum()

0

In [13]:
# # Check for duplicated rows
# df_train_features.T.duplicated().sum()

# # It is possible that two separate people responded exactly the same way
# # duplicated entries should not be removed

### To begin, I will focuse on the numerical columns and then come back and address the categorical ###

Start by setting up a new df df_train_num

In [14]:
df_train_num = df_train_features.select_dtypes('float64')

In [15]:
df_train_num.isna().mean()

h1n1_concern                   0.003445
h1n1_knowledge                 0.004343
behavioral_antiviral_meds      0.002658
behavioral_avoidance           0.007788
behavioral_face_mask           0.000711
behavioral_wash_hands          0.001573
behavioral_large_gatherings    0.003258
behavioral_outside_home        0.003070
behavioral_touch_face          0.004793
doctor_recc_h1n1               0.080878
doctor_recc_seasonal           0.080878
chronic_med_condition          0.036358
child_under_6_months           0.030704
health_worker                  0.030104
health_insurance               0.459580
opinion_h1n1_vacc_effective    0.014640
opinion_h1n1_risk              0.014528
opinion_h1n1_sick_from_vacc    0.014790
opinion_seas_vacc_effective    0.017299
opinion_seas_risk              0.019246
opinion_seas_sick_from_vacc    0.020107
household_adults               0.009323
household_children             0.009323
dtype: float64

#### health_insurance ####
For this column, as there are such a high percentage of null values it would bias the model too much to impute.  We will create a value -1 to indicate no response

In [16]:
df_train_num['health_insurance'].value_counts()

1.0    12697
0.0     1736
Name: health_insurance, dtype: int64

In [17]:
# Simply fillna with -1
df_train_num['health_insurance'].fillna(-1,inplace=True)

In [18]:
# Check counts again
df_train_num['health_insurance'].value_counts()

 1.0    12697
-1.0    12274
 0.0     1736
Name: health_insurance, dtype: int64

The rest of the columns have such low percentage of null values that for now I will utilize the SimpleImputer to impute the mean of the column for these

### Test Feature Cleanup ###

Repeat process with test features

In [19]:
df_test_num = df_test_features.select_dtypes('float64')

In [20]:
# Simply fillna with -1
df_test_num['health_insurance'].fillna(-1,inplace=True)

In [21]:
X_remainder = df_train_num
X_test = df_test_num

### Split provided training data for optimization ###

In [22]:
y_remainder = df_train_labels.drop('respondent_id',axis=1)

In [23]:
# from data.multilabel import multilabel_train_test_split

X_train, X_val, y_train, y_val = multilabel_train_test_split(X_remainder, y_remainder, size=0.25)

In [24]:
len(X_train)

20031

In [25]:
len(X_val)

6676

In [26]:
from sklearn.feature_selection import chi2, SelectKBest

from sklearn.pipeline import Pipeline, FeatureUnion

from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import roc_auc_score

from sklearn.model_selection import GridSearchCV

In [27]:
# # Default one vs rest logistic regression model
# logreg = OneVsRestClassifier(LogisticRegression()).fit(X_train,y_train)
# print(f'Training score: {logreg.score(X_train,y_train)}')
# print(f'Test score: {logreg.score(X_val,y_val)}')

In [28]:
# Set up the scaler and model pipeline
estimators = [
        ('imputer', SimpleImputer()),
        ('int', SparseInteractions(degree=2)),
        ('scale', MaxAbsScaler()),
        ('clf', OneVsRestClassifier(LogisticRegression()))
]

# Instantiate pipeline
pipe = Pipeline(estimators)

# Fit it to the training data
pipe.fit(X_train,y_train)

# Get predictions for roc_auc_score
predictions = pipe.predict_proba(X_val)

# Print train and test scores
print(f'Training score: {pipe.score(X_train,y_train)}')
print(f'Test score: {pipe.score(X_val,y_val)}')
print(f'Test roc_auc_score: {roc_auc_score(y_val,predictions)}')

Training score: 0.677949178772902
Test score: 0.6662672258837627
Test roc_auc_score: 0.841657310267617


# Predict holdout set and write submission

Finally, we want to use our trained pipeline to predict the holdout dataset. We will write our predictions to a file, `predictions.csv`, that we can submit on [DrivenData](http://www.drivendata.org)!

In [29]:
# Recall the test data was loaded, prepared and saved as X_test

# Make predictions
predictions = pipe.predict_proba(X_test)

# Format correctly in new DataFrame: prediction_df
prediction_df = pd.DataFrame(columns=pd.get_dummies(y_train).columns,
                             index=X_test.index,
                             data=predictions)

# Display the results
display(prediction_df)

,h1n1_vaccine,seasonal_vaccine
0,0.133706,0.427298
1,0.059097,0.119857
2,0.174763,0.648489
3,0.666805,0.898690
4,0.688374,0.763070
...,...,...
26703,0.218514,0.492566
26704,0.064795,0.254118
26705,0.207189,0.357649
26706,0.016446,0.274926


In [30]:
# Grab the respondent_ids for the test set from before
df_test_ids

,respondent_id
0,26707
1,26708
2,26709
3,26710
4,26711
...,...
26703,53410
26704,53411
26705,53412
26706,53413


In [31]:
# Merge the final prediction datatframe
final_df = pd.DataFrame(df_test_ids.merge(prediction_df,left_index=True,right_index=True),index=df_test_ids.index)
final_df

,respondent_id,h1n1_vaccine,seasonal_vaccine
0,26707,0.133706,0.427298
1,26708,0.059097,0.119857
2,26709,0.174763,0.648489
3,26710,0.666805,0.898690
4,26711,0.688374,0.763070
...,...,...,...
26703,53410,0.218514,0.492566
26704,53411,0.064795,0.254118
26705,53412,0.207189,0.357649
26706,53413,0.016446,0.274926


In [32]:
# Save prediction_df to csv called "predictions.csv"
final_df.to_csv("predictions.csv",index=False)

## sklearn.multioutput ##

In [33]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multioutput import ClassifierChain

In [34]:
# Set up the scaler and model pipeline
estimators = [
        ('imputer', SimpleImputer()),
        ('int', SparseInteractions(degree=2)),
        ('scale', MaxAbsScaler()),
        ('clf', ClassifierChain(LogisticRegression()))
]

# Instantiate pipeline
chain_pipe = Pipeline(estimators)

# Fit it to the training data
chain_pipe.fit(X_train,y_train)

# Get predictions for roc_auc_score
chain_pred = chain_pipe.predict_proba(X_val)

# Print train and test scores
print(f'Training score: {chain_pipe.score(X_train,y_train)}')
print(f'Test score: {chain_pipe.score(X_val,y_val)}')
print(f'Test roc_auc_score: {roc_auc_score(y_val,chain_pred)}')

Training score: 0.682192601467725
Test score: 0.6682144997004195
Test roc_auc_score: 0.8395233015633579


In [35]:
# Recall the test data was loaded, prepared and saved as X_test

# Make predictions
chain_predictions = chain_pipe.predict_proba(X_test)

# Format correctly in new DataFrame: prediction_df
chain_prediction_df = pd.DataFrame(columns=pd.get_dummies(y_train).columns,
                             index=X_test.index,
                             data=chain_predictions)

# Merge the final prediction datatframe
final_df = pd.DataFrame(df_test_ids.merge(chain_prediction_df,left_index=True,right_index=True),index=df_test_ids.index)

# Save prediction_df to csv called "predictions.csv"
final_df.to_csv("chain_predictions.csv",index=False)

## XGBoost ##

In [36]:
from xgboost import XGBClassifier

In [37]:
# Set up the scaler and model pipeline
estimators = [
        ('imputer', SimpleImputer()),
        ('int', SparseInteractions(degree=2)),
        ('scale', MaxAbsScaler()),
        ('clf', XGBClassifier(objective='multi:softmax',num_classes=2))
]

# Instantiate pipeline
xgb_pipe = Pipeline(estimators)

# Fit it to the training data
xgb_pipe.fit(X_train,y_train)

# Get predictions for roc_auc_score
xgb_preds = xgb_pipe.predict_proba(X_val)

# Print train and test scores
print(f'Training score: {xgb_pipe.score(X_train,y_train)}')
print(f'Test score: {xgb_pipe.score(X_val,y_val)}')
print(f'Test roc_auc_score: {roc_auc_score(y_val,xgb_preds)}')

ValueError: y should be a 1d array, got an array of shape (20031, 2) instead.